# Dependency Version Checker

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://files.realpython.com/media/Managing-Python-Dependencies_Watermarked.0f9e02d1860f.jpg"> 
</p>
</div>


## Description:

This application reads a `requirements.txt` file, retrieves the latest available versions of the listed dependencies, and checks whether it is safe to update them based on major and minor version rules. It then displays the results in a markdown table.  

### Key Points:  

- Reads dependencies from `requirements.txt`, filtering out comments.  

- Fetches the latest version of each package from PyPI.  

- Compares versions to determine if an update is safe (same major version and minor version upgrade allowed).  

- Generates a markdown table summarizing the results.  

- Displays the table with package names, current versions, latest versions, and update safety status.  


## Step 1: Extract and Display Dependencies from requirements.txt


- Opens the `requirements.txt` file in read mode.  

- Reads all lines and filters out comments while keeping only dependencies with specified versions.  

- Strips unnecessary whitespace and stores the dependencies in a list.  

- Prints the extracted dependencies.  


In [ ]:
with open ("requirements.txt", "r") as file: 

    dependencies = [
    line.strip() for line in file.readlines()
    if '==' in line and not line.strip().startswith('#')
]

print ("Dependencies found: ", dependencies)

## Step 2: Fetch the Latest Package Version from PyPI  

- Constructs the URL for the package’s metadata on PyPI.  

- Sends a request to fetch the package details in JSON format.  

- Extracts and returns the latest version from the response.  

- Handles errors (e.g., network issues or missing package data) and returns `"Unknown Package"` if the request fails.  


In [ ]:
import requests

def get_latest_version(package):
    """
    Fetches the latest available version of a package from PyPI.

    Args:
    - package (str): The name of the package to check.

    Returns:
    - str: The latest version as a string if successful, otherwise "Unknown Package".
    
    Handles:
    - Network errors (e.g., connection issues, timeouts).
    - Invalid responses or missing data from PyPI.
    """

    try:
        url = f"https://pypi.org/pypi/{package}/json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data['info']['version']
    except (requests.RequestException, KeyError):
        return("Unknown Package")


## Step 3: Checking if Updating a Dependency is Safe  

- Parses the current and latest version strings into `Version` objects.  

- Catches invalid version formats to prevent errors.  

- Determines if the update is safe by ensuring:  

  - The major version remains the same (avoiding breaking changes).  

  - The minor version of the latest release is greater than or equal to the current version.  

- Returns `True` if the update is safe, otherwise `False`.  


In [ ]:
from packaging.version import Version, InvalidVersion

def is_safe_to_update(current, latest):
    """
    Determines whether updating a package is safe based on versioning rules.

    A package update is considered safe if:
    - The major version remains the same (avoiding potential breaking changes).
    - The minor version of the latest release is greater than or equal to the current version.

    Args:
    - current (str): The current version of the package.
    - latest (str): The latest available version of the package.

    Returns:
    - bool: True if the update is safe, False otherwise.
    """
    try: 
        current_ver = Version(current)
        latest_ver = Version(latest)
    except InvalidVersion:
        return False

    return (current_ver.major == latest_ver.major) and (current_ver.minor <= latest_ver.minor)


# Step 4: Generating and Displaying Dependency Update Status    

- Iterates through the extracted dependencies.  

- Splits each dependency into package name and version.  

- Fetches the latest version from PyPI.  

- Checks if updating is safe using the version comparison function.  

- Stores the results in a structured list.  

- Generates a Markdown table displaying:  

  - **Package name**  
  
  - **Current version**
    
  - **Latest version**  
  
  - **Whether it's safe to update** ✅⚠️  
  

- Displays the table using IPython’s Markdown renderer.  


In [ ]:
from IPython.display import Markdown, display

results = []

for dep in dependencies:
    try:
        package, current_version = dep.split('==')
    except ValueError:
        print(f"Skipping invalid entry: {dep}")
        continue

    latest_version = get_latest_version(package)
    safe = "✅ Yes" if latest_version != "Unknown" and is_safe_to_update(current_version, latest_version) else "⚠️ No"
    results.append((package, current_version, latest_version, safe))

def generate_markdown_table(results):
    """
    Generates a markdown-formatted table summarizing package version information.

    Args:
    - results (list of tuples): Each tuple contains (package, current_version, latest_version, safety_status).

    Returns:
    - str: A formatted markdown table.
    """
    if not results:
        return "No valid dependencies found."  
    header = "| Package | Current Version | Latest Version | Safe to Update? |"
    separator = "| --- | --- | --- | --- |"
    rows = "\n".join([f"| {package} | {current_version} | {latest_version} | {safe} |" for package, current_version, latest_version, safe in results])
    return f"{header}\n{separator}\n{rows}"

markdown_table = generate_markdown_table(results)

display(Markdown(markdown_table))


## Conclusion:

The **Dependency Version Checker** provides a simple yet effective way to manage Python package updates.  

By:  
- Extracting dependencies from `requirements.txt`.  

- Fetching the latest versions from PyPI.  

- Assessing update safety.  

It helps developers maintain up-to-date and stable environments.  

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

